In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
years = list(range(2023,2021, -1))
years

[2023, 2022]

In [3]:
all_matches = []
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [26]:
import time
for year in years:
    data = requests.get(standing_url)
    soup = BeautifulSoup(data.text)
    standing_table = soup.select("table.stats_table")[0]
    
    # Get the full URL of each Squard
    links = standing_table.find_all("a")
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standing_url = f"https://fbref.com/{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
        
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match = "Scores & Fixtures")[0]
        
        soup = BeautifulSoup(data.text)
        links_shooting = soup.find_all("a")
        links_shooting = [l.get("href") for l in links_shooting]
        links_shooting = [l for l in links_shooting if l and "all_comps/shooting/" in l]
        data_shooting = requests.get(f"https://fbref.com{links_shooting[0]}")

        try:
            shooting = pd.read_html(data_shooting.text, match = "Shooting")[0]
            shooting.columns = shooting.columns.droplevel()
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist","FK", "PK", "PKatt"]], on = "Date")
        except ValueError:
            continue
            
        links_passing = soup.find_all('a')
        links_passing = [l.get("href") for l in links_passing]
        links_passing = [l for l in links_passing if l and "all_comps/passing" in l]
        data_passing = requests.get(f"https://fbref.com{links_passing[0]}")
        
        try:
            passing = pd.read_html(data_passing.text, match = "Passing")[0]
            passing.columns = passing.columns.droplevel()
            passing = passing.loc[:,~passing.columns.duplicated()]
            team_data = team_data.merge(passing[["Date","Cmp%"]], on = "Date")
        except ValueError:
            continue
            
        links_possession = soup.find_all("a")
        links_possession = [l.get("href") for l in links_possession]
        links_possession = [l for l in links_possession if l and "all_comps/possession" in l]
        data_possession = requests.get(f"https://fbref.com{links_possession[0]}")
        
        try:
            possession = pd.read_html(data_possession.text, match = "Possession")[0]
            possession.columns = possession.columns.droplevel()
            team_data = team_data.merge(possession[["Date","Touches","Def Pen","Def 3rd","Mid 3rd","Att 3rd","Att Pen", "Succ%","Rec","Prog"]], on = "Date")
        except ValueError:
            continue
        
        links_misc = soup.find_all("a")
        links_misc = [l.get("href") for l in links_misc]
        links_misc = [l for l in links_misc if l and "all_comps/misc" in l]
        data_misc = requests.get(f"https://fbref.com{links_misc[0]}")
        
        try:
            misc = pd.read_html(data_misc.text, match = "Misc")[0]
            misc.columns = misc.columns.droplevel()
            team_data = team_data.merge(misc[["Date","Won%"]], on = "Date")
        except ValueError:
            continue
        
            
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [27]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("matches.csv")
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,cmp%,touches,def pen,def 3rd,mid 3rd,att 3rd,att pen,succ%,rec,prog
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2019-04-13,12:30,Premier League,Matchweek 34,Sat,Away,L,0,4,Tottenham,...,70.9,434.0,53.0,143.0,211.0,87.0,15.0,61.1,237.0,14.0
36,2019-04-20,15:00,Premier League,Matchweek 35,Sat,Home,L,1,2,Watford,...,79.2,524.0,49.0,142.0,264.0,124.0,12.0,58.3,322.0,25.0
37,2019-04-26,20:00,Premier League,Matchweek 36,Fri,Away,L,0,5,Liverpool,...,67.4,451.0,55.0,167.0,198.0,90.0,10.0,33.3,225.0,7.0
38,2019-05-05,14:00,Premier League,Matchweek 37,Sun,Home,D,1,1,Manchester Utd,...,68.9,471.0,68.0,172.0,191.0,115.0,13.0,35.7,254.0,15.0
